In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import measure, segmentation, feature
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer
from scipy.ndimage import zoom
from skimage.morphology import skeletonize, skeletonize_3d

from skimage import filters, morphology

from scipy.ndimage.filters import convolve, correlate
from scipy import signal

from skimage.filters import frangi, sato

from PIL import Image
import pickle

In [2]:
source_dir = './data/*'
files = list(sorted(glob.glob(source_dir + '/*.raw')))
list(enumerate(files))

[(0, './data/P01/P01_60um_1612x623x1108.raw'),
 (1, './data/P02/P02_60um_1387x778x1149.raw'),
 (2, './data/P03/P03_60um_1473x1163x1148.raw'),
 (3, './data/P04/P04_60um_1273x466x1045.raw'),
 (4, './data/P05/P05_60um_1454x817x1102.raw'),
 (5, './data/P06/P06_60um_1425x564x1028.raw'),
 (6, './data/P07/P7_60um_1216x692x926.raw'),
 (7, './data/P08/P08_60um_1728x927x1149.raw'),
 (8, './data/P09/P09_60um_1359x456x1040.raw'),
 (9, './data/P10/P10_60um_1339x537x1035.raw'),
 (10, './data/P11/P11_60um_1735x595x1150.raw'),
 (11, './data/P12/P12_60um_1333x443x864.raw'),
 (12, './data/P13/P13_60um_1132x488x877.raw')]

In [3]:
%%time
mask = load_volume(files[11], scale=0.5)
mask.shape

CPU times: user 1.58 s, sys: 1.04 s, total: 2.61 s
Wall time: 3.13 s


(432, 222, 666)

In [4]:
threshold = 70
mask = mask > threshold
# VolumeVisualizer(mask, binary=True).visualize()

## Utility functions

### Visualisation functions

In [2]:
def visualize_addition(base, base_with_addition):
    base = (base.copy() > 0).astype(np.uint8)
    addition = (base_with_addition > 0).astype(np.uint8)
    addition[base == 1] = 0
    ColorMapVisualizer(base + addition * 2).visualize()
    
def visualize_lsd(lsd_mask):
    ColorMapVisualizer(lsd_mask.astype(np.uint8)).visualize()
    
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()
    
def visualize_skeleton(mask, visualize_mask=True, visualize_both_versions=False):
    skeleton = skeletonize((mask > 0).astype(np.uint8))
    if not visualize_mask or visualize_both_versions:
        VolumeVisualizer(skeleton, binary=True).visualize()
    if visualize_mask or visualize_both_versions:
        skeleton = skeleton.astype(np.uint8) * 4
        mask = (mask > 0).astype(np.uint8) * 3
        mask[skeleton != 0] = 0
        ColorMapVisualizer(skeleton + mask).visualize()

def visualize_ultimate(lsd, base_mask):
    visualize_lsd(lsd)
    visualize_mask_non_bin(lsd)
    visualize_addition(base_mask, lsd)
    visualize_skeleton(lsd, visualize_mask=True)

In [3]:
def load_lsd_trees(filename):
    with open(filename, 'rb') as f:
        lsd_trees = pickle.load(f)
    return lsd_trees

def save_lsd_trees(lsd_trees, filename):
    with open(filename, 'wb') as f:
        pickle.dump(lsd_trees, f)

In [4]:
def spherical_kernel(outer_radius, thickness=1, filled=True):    
    outer_sphere = morphology.ball(radius=outer_radius)
    if filled:
        return outer_sphere
    
    inner_radius = outer_radius - thickness
    inner_sphere = morphology.ball(radius=inner_radius)
    
    begin = outer_radius - inner_radius
    end = begin + inner_sphere.shape[0]
    outer_sphere[begin:end, begin:end, begin:end] -= inner_sphere
    return outer_sphere

def convolve_with_ball(img, ball_radius, dtype=np.uint16, normalize=True):
    kernel = spherical_kernel(ball_radius, filled=True)
    convolved = signal.convolve(img.astype(dtype), kernel.astype(dtype), mode='same')
    
    if not normalize:
        return convolved
    
    return (convolved / kernel.sum()).astype(np.float16)


def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions_masks = []
    regions_labels = []
    bounding_boxes = []
    
    for props in region_props:
        if props.area >= min_size:
            main_regions_masks.append(props.filled_image)
            regions_labels.append(props.label)
            bounding_boxes.append(props.bbox)
            
    return main_regions_masks, regions_labels, bounding_boxes

In [5]:
def annihilate_jemiolas(mask, kernel_sizes=[10, 9, 8], fill_threshold=0.5, iters=1, conv_dtype=np.uint16):

    kernel_sizes_maps = []
    mask = mask.astype(np.uint8)
    
    for i in range(iters):
        kernel_size_map = np.zeros(mask.shape, dtype=np.uint8)
        best_fill_percentage = np.zeros(mask.shape, dtype=np.float16)

        for kernel_size in kernel_sizes:
            fill_percentage = convolve_with_ball(mask, kernel_size, dtype=conv_dtype, normalize=True)
            better_fill_indices = fill_percentage > best_fill_percentage
            kernel_size_map[better_fill_indices] = kernel_size
            best_fill_percentage[better_fill_indices] = fill_percentage[better_fill_indices]
            print(f'Iteration {i + 1} kernel {kernel_size} done')

        kernel_size_map *= best_fill_percentage > fill_threshold
        kernel_sizes_maps.append(kernel_size_map)
        mask = np.minimum(kernel_size_map + mask, 1).astype(np.uint8)        
        print(f'Iteration {i + 1} ended successfully')

    return kernel_sizes_maps

In [15]:
def annihilate_jemiolas_faster(mask, kernel_sizes=[10, 9, 8], fill_threshold=0.5, iters=1, conv_dtype=np.uint16):

    kernel_sizes_maps = []
    mask = mask.astype(np.uint8)
    
    for i in range(iters):
        kernel_size_map = np.zeros(mask.shape, dtype=np.uint8)

        for kernel_size in kernel_sizes:
            fill_percentage = convolve_with_ball(mask, kernel_size, dtype=conv_dtype, normalize=True)
            
            above_threshold_fill_indices = fill_percentage > fill_threshold
            kernel_size_map[above_threshold_fill_indices] = kernel_size + 1

            mask[above_threshold_fill_indices] = 1
            
            print(f'Iteration {i + 1} kernel {kernel_size} done')

        kernel_sizes_maps.append(kernel_size_map)
        print(f'Iteration {i + 1} ended successfully')

    return kernel_sizes_maps

def onion_kernels(mask, kernel_sizes=[10, 9, 8], fill_threshold=0.8, conv_dtype=np.uint16):

    mask = mask.astype(np.uint8)
    
    kernel_size_map = np.zeros(mask.shape, dtype=np.uint8)

    for kernel_size in sorted(kernel_sizes):
        fill_percentage = convolve_with_ball(mask, kernel_size, dtype=conv_dtype, normalize=True)
        above_threshold_fill_indices = fill_percentage >= fill_threshold
        kernel_size_map[above_threshold_fill_indices] = kernel_size + 1
        print(f'Kernel {kernel_size} done')

    return kernel_size_map

## Main region extraction

In [10]:
mask_main = get_main_regions(mask)[0][0].astype(np.uint8)
# VolumeVisualizer(mask_main, binary=True).visualize()
# VolumeVisualizer(skeletonize_3d(mask_main.astype(np.uint8)), binary=True).visualize()

In [18]:
lsd_trees = annihilate_jemiolas_faster(mask_main, kernel_sizes=range(0, 13), iters=3)

Iteration 1 kernel 0 done
Iteration 1 kernel 1 done
Iteration 1 kernel 2 done
Iteration 1 kernel 3 done
Iteration 1 kernel 4 done
Iteration 1 kernel 5 done
Iteration 1 kernel 6 done
Iteration 1 kernel 7 done
Iteration 1 kernel 8 done
Iteration 1 kernel 9 done
Iteration 1 kernel 10 done
Iteration 1 kernel 11 done
Iteration 1 kernel 12 done
Iteration 1 ended successfully
Iteration 2 kernel 0 done
Iteration 2 kernel 1 done
Iteration 2 kernel 2 done
Iteration 2 kernel 3 done
Iteration 2 kernel 4 done
Iteration 2 kernel 5 done
Iteration 2 kernel 6 done
Iteration 2 kernel 7 done
Iteration 2 kernel 8 done
Iteration 2 kernel 9 done
Iteration 2 kernel 10 done
Iteration 2 kernel 11 done
Iteration 2 kernel 12 done
Iteration 2 ended successfully
Iteration 3 kernel 0 done
Iteration 3 kernel 1 done
Iteration 3 kernel 2 done
Iteration 3 kernel 3 done
Iteration 3 kernel 4 done
Iteration 3 kernel 5 done
Iteration 3 kernel 6 done
Iteration 3 kernel 7 done
Iteration 3 kernel 8 done
Iteration 3 kernel 9 d

In [13]:
visualize_skeleton(lsd_trees[-1])

In [7]:
# save_lsd_trees(lsd_trees, './trash/P12-lsd-trees')
lsd_trees = load_lsd_trees('./trash/P12-lsd-trees')
kozak_mask = (lsd_trees[-1] > 0).astype(np.uint8)
visualize_mask_bin(kozak_mask)

In [14]:
sklt = skeletonize_3d(kozak_mask)
sklt.sum()

25564

In [16]:
%%time
onion = onion_kernels(kozak_mask, kernel_sizes=range(12), fill_threshold=0.8)

Kernel 0 done
Kernel 1 done
Kernel 2 done
Kernel 3 done
Kernel 4 done
Kernel 5 done
Kernel 6 done
Kernel 7 done
Kernel 8 done
Kernel 9 done
Kernel 10 done
Kernel 11 done
CPU times: user 1min 30s, sys: 28 s, total: 1min 58s
Wall time: 2min 9s


In [17]:
visualize_lsd(onion)

In [51]:
def correct_skeleton(skeleton, kernel_size_map):
    max_radius = int(kernel_size_map.max())
    padded_skeleton = np.pad(skeleton, max_radius)
    padded_kernel_map = np.pad(kernel_size_map, max_radius)
    
    skeleton_voxels = np.argwhere(padded_skeleton)
    kernels = [spherical_kernel(radius) for radius in range(max_radius)]
    
    new_skeleton = np.zeros(padded_skeleton.shape)
    
    for voxel_coords in skeleton_voxels:
        x, y, z = tuple(voxel_coords)
        kernel_radius = padded_kernel_map[x, y, z] - 1
        kernel = kernels[kernel_radius]
        
        kernel_x, kernel_y, kernel_z = tuple(voxel_coords - kernel_radius)
        kernel_diameter = 2 * kernel_radius + 1
        kernel_map_slice = padded_kernel_map[
            kernel_x:kernel_x + kernel_diameter,
            kernel_y:kernel_y + kernel_diameter,
            kernel_z:kernel_z + kernel_diameter
        ]
        
        neighbours = kernel_map_slice * kernel
        
        if neighbours.max() == neighbours[kernel_radius, kernel_radius, kernel_radius]:
            target_voxel = (x, y, z)
            
        else:
            local_max_coords = np.argwhere(neighbours == neighbours.max())[0]
            dx, dy, dz = tuple(local_max_coords - kernel_radius)
            target_voxel = (x + dx, y + dy, z + dz)
        
        new_skeleton[target_voxel] = 1
        
    return new_skeleton[max_radius:-max_radius, max_radius:-max_radius, max_radius:-max_radius]


def propagate_thiccness(skeleton, kernel_size_map):
    padded_skeleton = np.pad(skeleton, 1)
    padded_kernels_map = np.pad(kernel_size_map, 1)
    
    thiccness_map = np.zeros(padded_kernels_map.shape)
    thiccness_map[padded_skeleton > 0] = padded_kernels_map[padded_skeleton > 0]
    
    queue = list([tuple(coords) for coords in np.argwhere(padded_skeleton)])
    while(len(queue) > 0):
        x, y, z = queue.pop(0)
        thiccness = thiccness_map[x, y, z]
        
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                for dz in [-1, 0, 1]:
                    neighbour_x = x + dx
                    neighbour_y = y + dy
                    neighbour_z = z + dz
                    if thiccness_map[neighbour_x, neighbour_y, neighbour_z] > 0:
                        continue
                        
                    if padded_kernels_map[neighbour_x, neighbour_y, neighbour_z] == 0:
                        continue
                        
                    thiccness_map[neighbour_x, neighbour_y, neighbour_z] = thiccness
                    queue.append((neighbour_x, neighbour_y, neighbour_z))
                        
    return thiccness_map[1:-1, 1:-1, 1:-1]

In [38]:
%%time

new_sklt = sklt
for i in range(5):
    new_sklt = correct_skeleton(new_sklt, onion)

CPU times: user 6.95 s, sys: 835 ms, total: 7.79 s
Wall time: 7.8 s


In [39]:
visualize_addition(new_sklt, sklt)

In [52]:
%%time
thicc_map = propagate_thiccness(new_sklt, onion)

CPU times: user 34.2 s, sys: 416 ms, total: 34.6 s
Wall time: 35 s


In [53]:
visualize_lsd(thicc_map)

In [21]:
XDD = (lsd_trees[-1] > 0).astype(np.uint8)
visualize_skeleton(XDD)
# sk = skeletonize_3d(XDD)
labels = measure.label(XDD, connectivity=3)
labels.max()
# visualize_mask_bin(labels == 1)

1

In [15]:
regions_data = get_main_regions(XDD)
main_rec = regions_data[0][0]
main_rec_label = regions_data[1][0]
main_rec_bbox = regions_data[2][0]

main_rec_label, main_rec_bbox

(2, (3, 2, 2, 411, 195, 645))

In [16]:
extra_regions = XDD.copy()
extra_regions[3:411, 2:195, 2:645] -= main_rec
visualize_addition(extra_regions, XDD)

# Fun

In [18]:
def central_annihilation(mask, kernel_sizes=[10, 9, 8], fill_threshold=0.8, conv_dtype=np.uint16):

    mask = mask.astype(np.uint8)
    
    kernel_size_map = np.zeros(mask.shape, dtype=np.uint8)

    for kernel_size in sorted(kernel_sizes):
        fill_percentage = convolve_with_ball(mask, kernel_size, dtype=conv_dtype, normalize=True)
        above_threshold_fill_indices = fill_percentage >= fill_threshold
        kernel_size_map[above_threshold_fill_indices] = kernel_size + 1
        print(f'Kernel {kernel_size} done')

    return kernel_size_map

In [39]:
# lsd_trees = load_lsd_trees('filled_trees/P01/lsd_1st_to_6tf_faster.trees')
# print(len(lsd_trees))
best_lsd_tree = lsd_trees[-1]
# lsd_trees = []
visualize_lsd(best_lsd_tree)

In [24]:
kernel_size_map = central_annihilation(best_lsd_tree > 0, range(18), fill_threshold=0.8)

Kernel 0 done
Kernel 1 done
Kernel 2 done
Kernel 3 done
Kernel 4 done
Kernel 5 done
Kernel 6 done
Kernel 7 done
Kernel 8 done
Kernel 9 done
Kernel 10 done
Kernel 11 done
Kernel 12 done
Kernel 13 done
Kernel 14 done
Kernel 15 done
Kernel 16 done
Kernel 17 done


In [36]:
avg_kernel_map = convolve_with_ball(kernel_size_map, ball_radius=4)

In [38]:
real_slim_shady = (kernel_size_map > avg_kernel_map * 1.4)

visualize_addition(real_slim_shady, kernel_size_map)

In [41]:
%%time
hogg = feature.hog(kernel_size_map)

CPU times: user 16.8 s, sys: 689 ms, total: 17.5 s
Wall time: 17.9 s


In [55]:
hogg.shape[0]

87318

In [56]:
8**3

512

In [54]:
np.prod(kernel_size_map.shape) / hogg.shape[0]

599.941008726723